In [1]:
import numpy as np
import math
import glob
import utils

from parse import load_ps
from pprint import pprint
from scipy.signal import medfilt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import DistanceMetric
from sklearn.metrics import classification_report

In [2]:
files = utils.files_in_order('poses_compressed/frontraise')
X_train_names, X_test_names = train_test_split(files, test_size=0.4, random_state=42)
y_train = utils.get_labels(X_train_names)
y_test = utils.get_labels(X_test_names)

In [3]:
def load_features(names):
    output1 = [] # List of torso-arm vectors
    for filename in names:
        ps = load_ps('poses_compressed/frontraise/'+filename)
        poses = ps.poses

        right_present = [1 for pose in poses 
                if pose.rshoulder.exists and pose.relbow.exists and pose.rwrist.exists]
        left_present = [1 for pose in poses
                if pose.lshoulder.exists and pose.lelbow.exists and pose.lwrist.exists]
        right_count = sum(right_present)
        left_count = sum(left_present)
        side = 'right' if right_count > left_count else 'left'

        if side == 'right':
            joints = [(pose.rshoulder, pose.relbow, pose.rwrist, pose.rhip, pose.neck) for pose in poses]
        else:
            joints = [(pose.lshoulder, pose.lelbow, pose.lwrist, pose.lhip, pose.neck) for pose in poses]

        # filter out data points where a part does not exist
        joints = [joint for joint in joints if all(part.exists for part in joint)]
        joints = np.array(joints)
        
        # Shoulder to hip    
        torso_vecs = np.array([(joint[0].x - joint[3].x, joint[0].y - joint[3].y) for joint in joints])
        # Arm
        arm_vecs = np.array([(joint[0].x - joint[2].x, joint[0].y - joint[2].y) for joint in joints])

        # normalize vectors
        torso_vecs = torso_vecs / np.expand_dims(np.linalg.norm(torso_vecs, axis=1), axis=1)
        arm_vecs = arm_vecs / np.expand_dims(np.linalg.norm(arm_vecs, axis=1), axis=1)
        
        angles = np.degrees(np.arccos(np.clip(np.sum(np.multiply(torso_vecs, arm_vecs), axis=1), -1.0, 1.0)))
        
        output1.append(angles.tolist())
    return output1

X_train_1 = load_features(X_train_names)
X_test_1 = load_features(X_test_names)
#features = load_features(files)

In [4]:
predictions = []
for example in range(len(X_test_names)):
    # Store the average distance to good and bad training examples
    f1_good, f1_bad = [[] for i in range(2)]
    
    # Compare distance of current test example with all training examples
    for i in range(len(X_train_1)):
        dist1 = utils.DTWDistance(X_train_1[i], X_test_1[example])
        if y_train[i]:
            f1_good.append(dist1)
        else:
            f1_bad.append(dist1)
    good_score = np.mean(f1_good)
    bad_score = np.mean(f1_bad)
    if good_score < bad_score:
        predictions.append(1)
    else:
        predictions.append(0)
print(classification_report(y_test, predictions, target_names=['correct', 'incorrect']))

             precision    recall  f1-score   support

    correct       1.00      1.00      1.00         6
  incorrect       1.00      1.00      1.00         6

avg / total       1.00      1.00      1.00        12

